In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#Section 1: Get All QB MVP Candidatates since 2004 (Last 20 Years) Clean Data so that it is Functional

#Use pd.concat to combine csv files of data. Two separate formats given for data so start by separating them into different data sets

#OLD FORMAT:
mvp_link = "voting_mvp/{}.csv"
years = range(2006,2022)
old_format = pd.read_csv('voting_mvp/2006.csv')
old_format['Year'] = 2006
drop_cols = ['-9999', 'Awards', 'Succ%', 'Lng', 'Rk', 'Team', 'A/G']
for year in years:
    new = pd.read_csv(mvp_link.format(year))
    new['Year'] = year
    old_format = pd.concat([old_format, new])


#MODERN FORMAT:
modern_format1 = pd.read_csv('voting_mvp/2022.csv')
modern_format1['Year'] = 2022
modern_format2 = pd.read_csv('voting_mvp/2023.csv')
modern_format2['Year'] = 2023
modern_format = pd.concat([modern_format1, modern_format2])


#COLUMN ADJUSTMENT: SET TO FIRST ROW
modern_format.columns = modern_format.iloc[0]
old_format.columns = old_format.iloc[0]

In [2]:
#Section 2: REMOVE ALL NON-MVP AREAS. ALLL WE WANT IS SHARES AND QBS AND YEARS (POS USED LATER AS WELL)
#Masking to Only Get QBS
mask_modern = modern_format['Pos'] == 'QB'
mask_old = old_format['Pos'] == 'QB'
old_format = old_format[mask_old]
modern_format = modern_format[mask_modern]


#BASICALLY MASKING AND CONCAT
old_format = old_format.rename(columns = {2006 : 'Year'})
modern_format = modern_format.rename(columns = {2022 : 'Year'})
modern_format = modern_format[['Pos', 'Player', 'Share', 'Year']]
old_format = old_format[['Pos', 'Player', 'Share', 'Year']]
training_mvps = pd.concat([old_format, modern_format])


In [3]:
#Section 3: Loading in Passing and Rushing Data. Minor Column Work
#PASSING CSV CREATION. LOAD IN ALL YEARS. ADD NEW YEAR COLUMN. REMOVE UNIMPORTANT COLUMNS. 

link = "passing_stats/{}.csv" 
years = range(2006, 2024)
passing = pd.read_csv('passing_stats/2006.csv')
passing['Year'] = 2006
drop_cols = ['Rk', 'Awards', '4QC', 'QBrec', 'Succ%', 'GWD', 'Player-additional', 'Yds.1', 'Sk%', ]
#print(passing.columns)
for year in years:
    new = pd.read_csv(link.format(year))
    new['Year'] = year
    passing = pd.concat([passing, new])
passing = passing.drop(columns = drop_cols)
mask_min_games = passing['G'] >= 10
passing = passing[mask_min_games]



#SAME AS PASS BUT WITH SLIGHTLY DIFFERENT COLUMNS AND NO MASKING 

rush_link = "rushing_stats/{}.csv"
years = range(2006,2024)
rush = pd.read_csv('rushing_stats/2006.csv')
rush['Year'] = 2006
drop_cols = ['-9999', 'Awards', 'Succ%', 'Lng', 'Rk', 'Team', 'A/G']
for year in years:
    new = pd.read_csv(rush_link.format(year))
    new['Year'] = year
    rush = pd.concat([rush, new])
rush = rush.drop(columns = drop_cols)


In [4]:
#Section 3: Minor Adjustments.

#ADJUST SHARES COLUMN. REMOVE % SYMBOL, CONVERT TO FLOAT, DIVIDE BY 100 SO THEY ARE BETWEEN 0 AND 1


training_mvps['Share'] = training_mvps['Share'].str.replace('%', '')


#Converting to floats, dividing by 100 so that they are between 0 and 1.
training_mvps['Share'] = training_mvps['Share'].astype(float)
training_mvps['Share'] = training_mvps['Share'] / 100



In [5]:
#Section 4: Masking for Rush and Passing

#Rush
mask_rush = rush['Pos'] == 'QB'
rush = rush[mask_rush]
mask_min_games2 = rush['G'] >= 10
rush = rush[mask_min_games2]
rush

#Passing
pos_mask = passing['Pos'] == 'QB'
passing = passing[pos_mask]
gmask = passing['G'] >= 10
passing = passing[gmask]

#Renaming Rush/Passing Columns
cols_pass = ['Player', 'Age', 'Team' , 'Pos', 'G', 'GS', 'Cmp', 'pAtt', 'Cmp%', 'pYds', 'pTD', 'pTD%', 'Int', 'Int%', 'p1D', 'pLng', 'pY/A'
            , 'AY/A', 'Y/C', 'Y/G', 'Rate', 'QBR', 'Sk', 'NY/A', 'ANY/A', 'Year']

passing.columns = cols_pass
cols_rush = ['Player', 'Age', 'Pos', 'G', 'GS', 'rAtt', 'rYds', 'rTD', 'r1D', 'rY/A', 'rY/G', 'rFmb', 'Year']
rush.columns = cols_rush



In [10]:
#Section 5: Prepare Rush and Passing Merge

overall = passing.merge(rush, how = 'outer')


#ADD MVP SHARES, FILL NAN WITH ZEROS. CREATE TRAINING CSV FOR LATER MACHINE LEARNING

training = overall.merge(training_mvps, how = 'outer')
training
training['Share'] = training['Share'].fillna(0)
training = training.dropna()

#FINAL TRAINING SET RETURNED AS CSV
training.to_csv('training.csv')
print(training.head())

          Player  Age Team Pos   G  GS    Cmp   pAtt  Cmp%    pYds  ...  \
0  Aaron Rodgers   25  GNB  QB  16  16  341.0  536.0  63.6  4038.0  ...   
1  Aaron Rodgers   26  GNB  QB  16  16  350.0  541.0  64.7  4434.0  ...   
2  Aaron Rodgers   27  GNB  QB  15  15  312.0  475.0  65.7  3922.0  ...   
3  Aaron Rodgers   28  GNB  QB  15  15  343.0  502.0  68.3  4643.0  ...   
4  Aaron Rodgers   29  GNB  QB  16  16  371.0  552.0  67.2  4295.0  ...   

   ANY/A  Year  rAtt   rYds  rTD   r1D  rY/A  rY/G  rFmb  Share  
0   6.64  2008  56.0  207.0  4.0  21.0   3.7  12.9  10.0   0.00  
1   7.47  2009  58.0  316.0  5.0  24.0   5.4  19.8  10.0   0.00  
2   7.50  2010  64.0  356.0  4.0  22.0   5.6  23.7   4.0   0.00  
3   9.39  2011  60.0  257.0  3.0  18.0   4.3  17.1   4.0   0.96  
4   7.33  2012  54.0  259.0  2.0  20.0   4.8  16.2   5.0   0.00  

[5 rows x 34 columns]


In [7]:
#Section 6: Making the Testing Data Set. Follows same trajectory just without the shares columns


#LOAD IN DATA SETS
passing_2024 = pd.read_csv('passing_stats/2024.csv')
rushing_2024 = pd.read_csv('rushing_stats/2024.csv')
passing_2024['Year'] = 2024
rushing_2024['Year'] = 2024



#DROP SAME COLUMNS AS BEFORE FOR PASSING AND RUSHING (SAME SIZE CSV AS TRAINING)
drop_cols = ['Rk', 'Awards', '4QC', 'QBrec', 'Succ%', 'GWD', 'Player-additional', 'Yds.1', 'Sk%', ]
passing_2024 = passing_2024.drop(columns = drop_cols)

drop_cols = ['-9999', 'Awards', 'Succ%', 'Lng', 'Rk', 'Team', 'A/G']
rushing_2024 = rushing_2024.drop(columns = drop_cols)


#USE SAME NAMING FORMAT AS TRAINING GROUP
cols_pass = ['Player', 'Age', 'Team' , 'Pos', 'G', 'GS', 'Cmp', 'pAtt', 'Cmp%', 'pYds', 'pTD', 'pTD%', 'Int', 'Int%', 'p1D',
             'pLng', 'pY/A', 'AY/A', 'Y/C', 'Y/G', 'Rate', 'QBR', 'Sk', 'NY/A', 'ANY/A', 'Year']
passing_2024.columns = cols_pass
cols_rush = ['Player', 'Age', 'Pos', 'G', 'GS', 'rAtt', 'rYds', 'rTD', 'r1D', 'rY/A', 'rY/G', 'rFmb', 'Year']
rushing_2024.columns = cols_rush

In [9]:
#Section 7: Masking and Final Testing Dataset

#MASK FOR ONLY QB RUSHING AND PASSING DATA
mask_rush = rushing_2024['Pos'] == 'QB'
rushing_2024 = rushing_2024[mask_rush]


pos_mask = passing_2024['Pos'] == 'QB'
passing_2024 = passing_2024[pos_mask]

min_cmp = passing_2024['Cmp'] > 40
passing_2024 = passing_2024[min_cmp]
#MERGE PASSING AND RUSHING DATA. DROP OUTLIERS WITH 0 PASSING YARDS OR 0 RUSHING YARDS
testing = passing_2024.merge(rushing_2024, how = 'outer')
testing = testing.dropna()


#Final Column Adjustment: Due to Bye Weeks Counting Stats (TDS, Yards, Cmp, etc need to be adjusted to account for games played differences)
#We will do so by simplying predicting that the QB continues to have the exact same average for the rest of the season.

adjust_cols = ['Cmp', 'pAtt', 'pYds', 'pTD', 'Int', 'p1D', 'Sk', 'rAtt', 'rYds', 'rTD' , 'r1D' , 'rFmb', 'GS']
#Round because you can't have half a TD or half a yard
for col in adjust_cols:
    testing[col] = round(testing[col] * (17 / testing['G']))

testing['G'] = 17

testing
#RETURN FINAL TESTING DATA SET FOR 2024 AS CSV
testing.to_csv('testing.csv')
print(testing.head())

               Player  Age Team Pos   G    GS    Cmp   pAtt  Cmp%    pYds  \
0       Aaron Rodgers   41  NYJ  QB  17  17.0  372.0  597.0  62.3  3980.0   
2         Andy Dalton   37  CAR  QB  17  14.0  300.0  453.0  66.3  2802.0   
3  Anthony Richardson   22  IND  QB  17  17.0  167.0  377.0  44.4  2714.0   
4      Baker Mayfield   29  TAM  QB  17  17.0  425.0  595.0  71.4  4513.0   
5              Bo Nix   24  DEN  QB  17  17.0  348.0  555.0  62.6  3311.0   

   ...  NY/A  ANY/A  Year   rAtt   rYds  rTD   r1D  rY/A  rY/G  rFmb  
0  ...  5.81   5.77  2024   23.0   81.0  0.0   6.0   3.6   4.8   4.0  
2  ...  5.70   4.92  2024   31.0   96.0  0.0   6.0   3.1   5.7   6.0  
3  ...  6.29   4.63  2024  116.0  686.0  3.0  48.0   5.9  40.3  17.0  
4  ...  6.70   6.86  2024   62.0  357.0  4.0  25.0   5.7  21.0   8.0  
5  ...  5.34   4.99  2024  110.0  557.0  8.0  51.0   5.1  32.8   0.0  

[5 rows x 33 columns]
